In [1]:
# Using pandas to process data
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline

from utils.data_util import parse_bj_aq_data, generate_model_data
from utils.plot_util import plot_station, plot_stations

%load_ext autoreload
%autoreload 2

# 数据解析

In [2]:
bj_aq_data, stations, bj_aq_stations, bj_aq_stations_merged = parse_bj_aq_data()

NaN in PM2.5 is 23459, 6.508615 %
NaN in PM10 is 96175, 26.683406 %
NaN in NO2 is 21720, 6.026135 %
NaN in CO is 46144, 12.802486 %
NaN in O3 is 23732, 6.584358 %
NaN in SO2 is 21664, 6.010598 %
There are 35 air quality stations in Beijing

The stations in Beijing are:
 {'fengtaihuayuan_aq', 'zhiwuyuan_aq', 'aotizhongxin_aq', 'tongzhou_aq', 'gucheng_aq', 'qianmen_aq', 'mentougou_aq', 'daxing_aq', 'miyunshuiku_aq', 'huairou_aq', 'tiantan_aq', 'yanqin_aq', 'dingling_aq', 'fangshan_aq', 'beibuxinqu_aq', 'nongzhanguan_aq', 'pingchang_aq', 'yungang_aq', 'dongsi_aq', 'wanshouxigong_aq', 'xizhimenbei_aq', 'nansanhuan_aq', 'wanliu_aq', 'yufa_aq', 'yizhuang_aq', 'yongledian_aq', 'yongdingmennei_aq', 'liulihe_aq', 'shunyi_aq', 'badaling_aq', 'donggaocun_aq', 'dongsihuan_aq', 'miyun_aq', 'guanyuan_aq', 'pinggu_aq'}


/notebooks/KDD_Cup_2018/utils/data_util.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  bj_aq_station.drop("utc_time", axis=1, inplace=True)
/notebooks/KDD_Cup_2018/utils/data_util.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  bj_aq_station.drop("stationId", axis=1, inplace=True)


In [39]:
bj_aq_stations_merged.head()

,dingling_aq_PM2.5,dingling_aq_PM10,dingling_aq_NO2,dingling_aq_CO,dingling_aq_O3,dingling_aq_SO2,nansanhuan_aq_PM2.5,nansanhuan_aq_PM10,nansanhuan_aq_NO2,nansanhuan_aq_CO,...,wanshouxigong_aq_NO2,wanshouxigong_aq_CO,wanshouxigong_aq_O3,wanshouxigong_aq_SO2,xizhimenbei_aq_PM2.5,xizhimenbei_aq_PM10,xizhimenbei_aq_NO2,xizhimenbei_aq_CO,xizhimenbei_aq_O3,xizhimenbei_aq_SO2
format_time,,,,,,,,,,,,,,,,,,,,,
2017-01-01 14:00:00,339.0,372.0,137.0,5.9,6.0,18.0,431.0,467.0,147.0,6.5,...,140.0,0.6,7.0,5.0,514.0,NaN,171.0,8.3,8.0,15.0
2017-01-01 15:00:00,343.0,372.0,137.0,5.4,4.0,19.0,426.0,444.0,144.0,6.7,...,139.0,6.1,6.0,5.0,508.0,NaN,161.0,0.8,8.0,15.0
2017-01-01 16:00:00,311.0,372.0,123.0,5.1,5.0,13.0,420.0,451.0,136.0,0.7,...,138.0,6.5,6.0,4.0,497.0,NaN,164.0,7.6,8.0,14.0
2017-01-01 17:00:00,256.0,372.0,118.0,4.1,3.0,17.0,417.0,446.0,128.0,7.4,...,133.0,6.9,7.0,5.0,500.0,NaN,155.0,7.8,8.0,14.0
2017-01-01 18:00:00,57.0,372.0,33.0,0.8,43.0,17.0,426.0,465.0,124.0,0.8,...,126.0,6.6,7.0,5.0,480.0,NaN,147.0,7.8,8.0,14.0


In [3]:
# 还有一些特征仍然存在缺失值
bj_aq_stations_merged.isnull().any()

miyunshuiku_aq_PM2.5       False
miyunshuiku_aq_PM10         True
miyunshuiku_aq_NO2         False
miyunshuiku_aq_CO          False
miyunshuiku_aq_O3          False
miyunshuiku_aq_SO2         False
wanshouxigong_aq_PM2.5     False
wanshouxigong_aq_PM10      False
wanshouxigong_aq_NO2       False
wanshouxigong_aq_CO        False
wanshouxigong_aq_O3        False
wanshouxigong_aq_SO2       False
pinggu_aq_PM2.5            False
pinggu_aq_PM10              True
pinggu_aq_NO2              False
pinggu_aq_CO               False
pinggu_aq_O3               False
pinggu_aq_SO2              False
guanyuan_aq_PM2.5          False
guanyuan_aq_PM10           False
guanyuan_aq_NO2            False
guanyuan_aq_CO             False
guanyuan_aq_O3             False
guanyuan_aq_SO2            False
yanqin_aq_PM2.5            False
yanqin_aq_PM10             False
yanqin_aq_NO2              False
yanqin_aq_CO               False
yanqin_aq_O3               False
yanqin_aq_SO2              False
          

In [4]:
# generate all training data at a time.
X_batches, Y_batches = generate_model_data(merged_data=bj_aq_stations_merged, m=10, X_hours=32, Y_hours=48, step=1)

In [5]:
print(len(X_batches), X_batches[0].shape, Y_batches[0].shape)

1012 (10, 32, 210) (10, 48, 210)


# 可视化

# 数据预处理
- 对缺失值进行了补全，使用 ffill

# 数据的下载补齐

- 空气质量数据
    - 10298 data for 35 stations
    - 2017:01:01 14:00 - 2018:03:31 15:00 

In [26]:
# 原有 aq 数据的范围
print(min(bj_aq_data.index), max(bj_aq_data.index))

2017-01-01 14:00:00 2018-03-31 15:00:00


In [30]:
# 新下载的 aq 数据
bj_aq_dataset_new = pd.read_csv("./KDD_CUP_2018/Beijing/aq/bj_airquality_2018-03-31-15-2018-04-15-23.csv")
bj_aq_dataset_new.head()

,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
0,2941450,dongsi_aq,2018-03-31 07:00:00,105.0,172.0,53.0,0.8,127.0,14.0
1,2941451,tiantan_aq,2018-03-31 07:00:00,95.0,123.0,54.0,0.9,121.0,15.0
2,2941452,guanyuan_aq,2018-03-31 07:00:00,95.0,139.0,66.0,0.8,123.0,13.0
3,2941453,wanshouxigong_aq,2018-03-31 07:00:00,99.0,172.0,58.0,1.0,119.0,14.0
4,2941454,aotizhongxin_aq,2018-03-31 07:00:00,117.0,166.0,67.0,0.8,111.0,14.0


In [31]:
length = bj_aq_dataset_new.shape[0]
formet_time = pd.Series([datetime.datetime.strptime(bj_aq_dataset_new["time"][i],'%Y-%m-%d %H:%M:%S') for i in range(length)])
bj_aq_dataset_new["format_time"] = formet_time
bj_aq_dataset_new.set_index("format_time", inplace=True)

In [33]:
# 新下载的 aq 数据的范围
print(min(bj_aq_dataset_new.index), max(bj_aq_dataset_new.index))

2018-03-31 07:00:00 2018-04-15 23:00:00


# ChangeLog
- 0419 对新下载的数据进行了数据探索
- TODO
    - 新下载的数据和之前的数据在时间轴上有重叠的部分，同时原始的数据有很大部分的缺失，可以使用新下载的数据对原始数据进行补齐
    - 分析之前的数据中，不同站点的时间轴缺失情况是怎么样的，如果有大段的缺失，建议丢弃该片段
        - 对于短期的 + 个别的缺失，使用插值的方式进行补全
        - 对于长期的 + 全特征的缺失，将对应的数据删除